In [2]:
import sys
# sys.path.insert(0, '../GET')


from Models.FCWithEvidences import graph_based_semantic_structure
from Fitting.FittingFC import char_man_fitter_query_repr1
import time
import json
from interactions import ClassificationInteractions
import matchzoo as mz
from handlers import cls_load_data
import argparse
import random
import numpy as np
import torch
import torch_utils
import numpy as np
import os
import datetime
from handlers.output_handler_FC import FileHandlerFC
from Evaluation import mzEvaluator as evaluator
from setting_keywords import KeyWordSettings
from matchzoo.embedding import entity_embedding
from Models.BiDAF.wrapper import GGNN, GGNN_with_GSL, Linear
from thirdparty.two_branches_attention import *


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset="Snopes"
fixed_length_left=30
fixed_length_right=100
log="logs/testget"
loss_type="cross_entropy"
batch_size=32
num_folds=5
use_claim_source=0
use_article_source=1
path="formatted_data/declare/"
hidden_size=300
epochs=100
num_att_heads_for_words=5
num_att_heads_for_evds=2
gnn_window_size=3
lr=0.0001
gnn_dropout=0.2
seed=123456
gsl_rate=0.6
fixed_length_left_src_chars = 20
fixed_length_right_src_chars = 20
seed = 123456
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [3]:
# dataset="PolitiFact"
# fixed_length_left=30
# fixed_length_right=100
# log="logs/testget"
# loss_type="cross_entropy"
# batch_size=32
# num_folds=5
# use_claim_source=1
# use_article_source=1
# path="formatted_data/declare/"
# hidden_size=300
# epochs=100
# num_att_heads_for_words=3
# num_att_heads_for_evds=1
# gnn_window_size=3
# lr=0.0001
# gnn_dropout=0.2
# seed=123456
# gsl_rate=0.6
# fixed_length_left_src_chars = 20
# fixed_length_right_src_chars = 20
# seed = 123456
# random.seed(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)

In [3]:
preprocessor = mz.preprocessors.CharManPreprocessor(fixed_length_left = fixed_length_left,
                                                    fixed_length_right = fixed_length_right,
                                                    fixed_length_left_src = fixed_length_left_src_chars,
                                                    fixed_length_right_src = fixed_length_right_src_chars)

In [4]:
dataset="Snopes"
fixed_length_left=30
fixed_length_right=100
log="logs/get"
batch_size=32
gnn_window_size=3
fold_idx=0
if not os.path.exists(log):
      os.mkdir(log)
secondary_log_folder = os.path.join(log, "log_results_%s" % (dataset))
if not os.path.exists(secondary_log_folder):
  os.mkdir(secondary_log_folder)
secondary_log_folder = secondary_log_folder
root = os.path.join(os.path.join(path,dataset), "mapped_data")
tx = time.time()
kfold_dev_results, kfold_test_results = [], []
list_metrics = KeyWordSettings.CLS_METRICS
outfolder_per_fold = os.path.join(secondary_log_folder, "Fold_%s" % fold_idx)
if not os.path.exists(outfolder_per_fold):
    os.mkdir(outfolder_per_fold)
logfolder_result_per_fold = os.path.join(outfolder_per_fold, "result_%s.txt" % int(seed))
file_handler = FileHandlerFC()
file_handler.init_log_files(logfolder_result_per_fold)
# root ="/content/drive/MyDrive/GET/formatted_data/declare/Snopes/mapped_data"
# ------------------------------------------------------------------------------------
predict_pack = cls_load_data.load_data(root + "/%sfold" % num_folds,  'test_%s' % 0, kfolds = num_folds)
train_pack = cls_load_data.load_data(root + "/%sfold" % num_folds, 'train_%sres' % 0, kfolds = num_folds)
# predict_pack = cls_load_data.load_data(root + "/%sfold" % num_folds,  'testnew' , kfolds = num_folds)
# train_pack = cls_load_data.load_data(root + "/%sfold" % num_folds, 'trainnew' , kfolds = num_folds)
valid_pack = cls_load_data.load_data(root, 'dev', kfolds = num_folds)

a = train_pack.left["text_left"].str.lower().str.split().apply(len).max()
b = valid_pack.left["text_left"].str.lower().str.split().apply(len).max()
c = predict_pack.left["text_left"].str.lower().str.split().apply(len).max()
max_query_length = max([a, b, c])
min_query_length = min([a, b, c])

a = train_pack.right["text_right"].str.lower().str.split().apply(len).max()
b = valid_pack.right["text_right"].str.lower().str.split().apply(len).max()
c = predict_pack.right["text_right"].str.lower().str.split().apply(len).max()
max_doc_length = max([a, b, c])
min_doc_length = min([a, b, c])

file_handler.myprint("Min query length, " + str(min_query_length) + " Min doc length " + str(min_doc_length))
file_handler.myprint("Max query length, " + str(max_query_length) + " Max doc length " + str(max_doc_length))
global additional_data
additional_data = {KeyWordSettings.OutputHandlerFactChecking: file_handler,
                    KeyWordSettings.GNN_Window: gnn_window_size}

print('parsing data')

train_processed = preprocessor.fit_transform(train_pack)  # This is a DataPack
valid_processed = preprocessor.transform(valid_pack)
predict_processed = preprocessor.transform(predict_pack)


train_interactions = ClassificationInteractions(train_processed, **additional_data)
valid_interactions = ClassificationInteractions(valid_processed, **additional_data)
predict_interactions = ClassificationInteractions(predict_processed, **additional_data)
file_handler.myprint('done extracting')
  

100%|██████████| 2756/2756 [00:00<00:00, 773728.37it/s]


Min query length, 22 Min doc length 100
Max query length, 28 Max doc length 100
parsing data


Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 3126/3126 [00:00<00:00, 10535.57it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 21074/21074 [00:07<00:00, 2966.63it/s]
Processing text_right with extend: 100%|██████████| 21074/21074 [00:00<00:00, 136865.73it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 3126/3126 [00:00<00:00, 11280.60it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 21074/21074 [00:07<00:00, 2711.95it/s]
Processing text_right with transform: 100%|██████████| 2756/2756 [00:00<00:00, 178439.69it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 433/433 [00:00<00:00, 11496.34it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2756/2756 [00:01<00:

Converting DataFrame to Normal Dictionary of Data
[NOTICE] MatchZoo use queryID and docID as index in dataframe left and right, therefore, iterrows will return index which is left_id or right_id



/Users/namle/Library/CloudStorage/GoogleDrive-hcmutepfl@gmail.com/My Drive/GET/interactions.py:15: RuntimeWarning: divide by zero encountered in power
  d_inv_sqrt = np.power(rowsum, -0.5).flatten()


[NOTICE] MatchZoo use queryID and docID as index in dataframe left and right, therefore, iterrows will return index which is left_id or right_id
Converting DataFrame to Normal Dictionary of Data
[NOTICE] MatchZoo use queryID and docID as index in dataframe left and right, therefore, iterrows will return index which is left_id or right_id
[NOTICE] MatchZoo use queryID and docID as index in dataframe left and right, therefore, iterrows will return index which is left_id or right_id
Converting DataFrame to Normal Dictionary of Data
[NOTICE] MatchZoo use queryID and docID as index in dataframe left and right, therefore, iterrows will return index which is left_id or right_id
[NOTICE] MatchZoo use queryID and docID as index in dataframe left and right, therefore, iterrows will return index which is left_id or right_id
done extracting


In [5]:
print("Loading word embeddings......")
t1_emb = time.time()
term_index = preprocessor.context['vocab_unit'].state['term_index']
glove_embedding = mz.datasets.embeddings.load_glove_embedding_FC(dimension = 300,
                                                                  term_index = term_index, **additional_data)

embedding_matrix = glove_embedding.build_matrix(term_index)
entity_embs1 = entity_embedding.EntityEmbedding(128)
claim_src_embs_matrix = entity_embs1.build_matrix(preprocessor.context['claim_source_unit'].state['term_index'])

entity_embs2 = entity_embedding.EntityEmbedding(128)
article_src_embs_matrix = entity_embs2.build_matrix(preprocessor.context['article_source_unit'].state['term_index'])

t2_emb = time.time()
print("Time to load word embeddings......", (t2_emb - t1_emb))

Loading word embeddings......
Word hit: (32823, 49881) 65.8026102123053
Time to load word embeddings...... 2.851768732070923


In [ ]:
embedding_matrix.shape

(49881, 300)

In [6]:
match_params = {}
match_params['embedding'] = embedding_matrix
match_params["num_classes"] = 2
match_params["fixed_length_right"] = 100
match_params["fixed_length_left"] = 30

# for claim source
match_params["use_claim_source"] = use_claim_source
match_params["claim_source_embeddings"] = claim_src_embs_matrix
# for article source
match_params["use_article_source"] = use_article_source
match_params["article_source_embeddings"] = article_src_embs_matrix
# multi-head attention
match_params["cuda"] = 0
match_params["num_att_heads_for_words"] = num_att_heads_for_words  # first level
match_params["num_att_heads_for_evds"] = num_att_heads_for_evds  # second level


match_params['dropout_gnn'] = 0.2
match_params["dropout_left"] = 0.2
match_params["dropout_right"] = 0.2
match_params["hidden_size"] = hidden_size

match_params["gsl_rate"] = 0.6

match_params["embedding_freeze"] = True
match_params["output_size"] = 2 # if args.dataset == "Snopes" else 3
match_model = graph_based_semantic_structure.Graph_basedSemantiStructure(match_params)
match_model.load_state_dict(torch.load("/Users/namle/Desktop/GET/snope_0",map_location=torch.device('cpu')))
import datetime
from handlers.output_handler_FC import FileHandlerFC
loss_type = 'cross_entropy'
epochs = 100
batch_size = 32
lr = 0.001
early_stopping = 10
outfolder_per_fold = "logs/get/log_results_PolitiFact/Fold_10"
curr_date = datetime.datetime.now().timestamp()
fixed_num_evidences = 30
file_handler = FileHandlerFC()
seed = 123456
fit_model = char_man_fitter_query_repr1.CharManFitterQueryRepr1(net = match_model, loss = loss_type, n_iter = epochs,
                                                  batch_size = batch_size, learning_rate = lr,
                                                  early_stopping = early_stopping, use_cuda = 0,
                                                  logfolder = outfolder_per_fold, curr_date = curr_date,
                                                  fixed_num_evidences = fixed_num_evidences,
                                                  output_handler_fact_checking = file_handler, seed=seed,
                                                  output_size=match_params["output_size"],args="args")

AttributeError: 'super' object has no attribute '__inigt__'

In [9]:
result_test= fit_model.evaluate(valid_interactions, 10)

In [10]:
result_test

{'auc': 0.8892635701076906,
 'f1_macro': 0.8164865437592711,
 'f1_micro': 0.8521939953810623,
 'f1': 0.7355371900826447,
 'precision_true_cls': 0.7063492063492064,
 'recall_true_cls': 0.7672413793103449,
 'f1_true_cls': 0.7355371900826447,
 'precision_false_cls': 0.9120521172638436,
 'recall_false_cls': 0.8832807570977917,
 'f1_false_cls': 0.8974358974358975,
 'precision_mixed_cls': 0.0,
 'recall_mixed_cls': 0.0,
 'f1_mixed_cls': 0.0}

In [421]:
size = 32
cuda = False
import interactions
import torch_utils as my_utils

# interactions: interactions.ClassificationInteractions
query_ind_idx =[]
query_ood_idx = []
import gc


def get_energy():
  query_ids, left_contents, left_lengths, query_sources, query_char_sources, query_adj, \
            evd_docs_ids, evd_docs_contents, evd_docs_lens, evd_sources, evd_cnt_each_query, evd_char_sources, \
            pair_labels, evd_docs_adj = fit_model._sampler.get_train_instances_char_man(predict_interactions,fit_model.fixed_num_evidences)

  for i in range(pair_labels.shape[0]):
    if pair_labels[i] == 1:
        query_ind_idx.append(i)
    else:
        query_ood_idx.append(i)
  logits = torch.tensor([])
  outputs = torch.tensor([])
  E = torch.tensor([])
  with torch.no_grad():

    for (minibatch_num,
                  (batch_query, batch_query_content, batch_query_len, batch_query_sources, batch_query_chr_src,
                    batch_query_adj, batch_evd_docs, batch_evd_contents, batch_evd_lens, batch_evd_sources,
                    # i.e. claim source
                    batch_evd_cnt_each_query, batch_evd_chr_src, batch_labels, batch_evd_docs_adj)) \
                      in enumerate(my_utils.minibatch(query_ids, left_contents, left_lengths, query_sources,
                                                      query_char_sources, query_adj,
                                                      evd_docs_ids, evd_docs_contents, evd_docs_lens, evd_sources,
                                                      evd_cnt_each_query, evd_char_sources, pair_labels, evd_docs_adj,
                                                      batch_size=32)):
        batch_query = my_utils.gpu(torch.from_numpy(batch_query), cuda)
        batch_query_content = my_utils.gpu(torch.from_numpy(batch_query_content), cuda)
        # batch_query_len = my_utils.gpu(torch.from_numpy(batch_query_len), self._use_cuda)
        batch_query_sources = my_utils.gpu(torch.from_numpy(batch_query_sources), cuda)
        batch_query_chr_src = my_utils.gpu(torch.from_numpy(batch_query_chr_src), cuda)
        batch_query_adj = my_utils.gpu(torch.from_numpy(batch_query_adj), cuda)

        batch_evd_docs = my_utils.gpu(torch.from_numpy(batch_evd_docs), cuda)
        batch_evd_contents = my_utils.gpu(torch.from_numpy(batch_evd_contents), cuda)
        # batch_evd_lens = my_utils.gpu(torch.from_numpy(batch_evd_lens), self._use_cuda)
        batch_evd_sources = my_utils.gpu(torch.from_numpy(batch_evd_sources), cuda)
        batch_evd_cnt_each_query = my_utils.gpu(torch.from_numpy(batch_evd_cnt_each_query), cuda)
        batch_evd_chr_src = my_utils.gpu(torch.from_numpy(batch_evd_chr_src), cuda)

        batch_labels = my_utils.gpu(torch.from_numpy(batch_labels), cuda)
        batch_evd_docs_adj = my_utils.gpu(torch.from_numpy(batch_evd_docs_adj), cuda)
        # total_pairs += self._batch_size * self.
        additional_data = {KeyWordSettings.EvidenceCountPerQuery: batch_evd_cnt_each_query,
                            KeyWordSettings.FCClass.QueryCharSource: batch_query_chr_src,
                            KeyWordSettings.FCClass.DocCharSource: batch_evd_chr_src,
                            KeyWordSettings.Query_Adj: batch_query_adj,
                            KeyWordSettings.Evd_Docs_Adj: batch_evd_docs_adj}
        n=30
        evd_count_per_query = batch_evd_cnt_each_query  # (B, )
        query_char_source = batch_query_chr_src
        doc_char_source = batch_evd_chr_src
        query_adj = batch_query_adj
        evd_docs_adj = batch_evd_docs_adj
        _, L = batch_query_content.size()
        batch_size = batch_query.size(0)
        # prunning at this step to remove padding\
        e_lens, e_conts, q_conts, q_lens, e_adj = [], [], [], [], []
        e_chr_src_conts = []
        expaned_labels = []
        for evd_cnt, q_cont, q_len, evd_lens, evd_doc_cont, evd_chr_src, label, evd_adj in \
                zip(evd_count_per_query, batch_query_content, batch_query_len,
                    batch_evd_lens, batch_evd_contents, doc_char_source, batch_labels, evd_docs_adj):
            evd_cnt = int(torch_utils.cpu(evd_cnt).detach().numpy())
            e_lens.extend(list(evd_lens[:evd_cnt]))
            e_conts.append(evd_doc_cont[:evd_cnt, :])  # stacking later
            e_adj.append(evd_adj[:evd_cnt])
            e_chr_src_conts.append(evd_chr_src[:evd_cnt, :])
            q_lens.extend([q_len] * evd_cnt)
            q_conts.append(q_cont.unsqueeze(0).expand(evd_cnt, L))
            expaned_labels.extend([int(torch_utils.cpu(label).detach().numpy())] * evd_cnt)
        # concat
        e_conts = torch.cat(e_conts, dim=0)  # (n1 + n2 + ..., R)
        e_chr_src_conts = torch.cat(e_chr_src_conts, dim=0)  # (n1 + n2 + ... , R)
        e_adj = torch.cat(e_adj, dim=0)     # (n1 + n2 + ..., R, R)
        e_lens = np.array(e_lens)  # (n1 + n2 + ..., )
        q_conts = torch.cat(q_conts, dim=0)  # (n1 + n2 + ..., R)
        q_lens = np.array(q_lens)
        assert q_conts.size(0) == q_lens.shape[0] == e_conts.size(0) == e_lens.shape[0]

        d_new_indices, d_old_indices = torch_utils.get_sorted_index_and_reverse_index(e_lens)
        e_lens = my_utils.gpu(torch.from_numpy(e_lens), cuda)
        x = batch_query_len
        q_new_indices, q_restoring_indices = torch_utils.get_sorted_index_and_reverse_index(x)
        x = my_utils.gpu(torch.from_numpy(x), cuda)
        # query_lens = my_utils.gpu(torch.from_numpy(query_lens), self._use_cuda)

        additional_paramters = {
            KeyWordSettings.Query_lens: x,  # 每一个query长度
            KeyWordSettings.Doc_lens: batch_evd_lens,
            KeyWordSettings.DocLensIndices: (d_new_indices, d_old_indices, e_lens),
            KeyWordSettings.QueryLensIndices: (q_new_indices, q_restoring_indices, x),
            KeyWordSettings.QuerySources: batch_query_sources,
            KeyWordSettings.DocSources: batch_evd_sources,
            KeyWordSettings.TempLabel: batch_labels,
            KeyWordSettings.DocContentNoPaddingEvidence: e_conts,
            KeyWordSettings.QueryContentNoPaddingEvidence: q_conts,
            KeyWordSettings.EvidenceCountPerQuery: evd_count_per_query,
            KeyWordSettings.FCClass.QueryCharSource: query_char_source,  # (B, 1, L)
            KeyWordSettings.FCClass.DocCharSource: e_chr_src_conts,
            KeyWordSettings.FIXED_NUM_EVIDENCES: n,
            KeyWordSettings.Query_Adj: query_adj,
            KeyWordSettings.Evd_Docs_Adj: e_adj                       # flatten->(n1 + n2 ..., R, R)
        }
        output,logit = match_model.predict(batch_query_content, batch_evd_contents, **additional_paramters)
        logits = torch.cat((logits,logit),dim = 0)
        outputs = torch.cat((outputs,output),dim=0)
        
  return logits,outputs
logits,outputs = get_energy()

# prob_logit = torch.cat((prob_logit,logits),dim = 0)
# energy_full = torch.cat((energy_full, energy_train), dim=0)

In [422]:
query_ids, left_contents, left_lengths, query_sources, query_char_sources, query_adj, \
            evd_docs_ids, evd_docs_contents, evd_docs_lens, evd_sources, evd_cnt_each_query, evd_char_sources, \
            pair_labels, evd_docs_adj = fit_model._sampler.get_train_instances_char_man(predict_interactions,fit_model.fixed_num_evidences)


In [341]:
outputs.size()

torch.Size([782, 3556])

In [321]:

from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")

In [342]:
id=list(range(outputs.size(0)))

In [332]:
vector_2_query = {}
for i,id in enumerate(query_ids):
    vector_2_query[i] = id

In [34]:
query_ids = [23,325,2,23,9,452,35,22,52,]

In [38]:
query_2_vector = {}
for i,id in enumerate(query_ids):
    query_2_vector[id] = i

In [ ]:
res = [query_2_vector]

In [39]:
query_2_vector([23,325,2,23])

TypeError: 'dict' object is not callable

In [37]:
vector_2_query[1]

NameError: name 'vector_2_query' is not defined

In [338]:
from qdrant_client.models import Distance, VectorParams

client.create_collection(
    collection_name="test_0",
    vectors_config=VectorParams(size=3556, distance=Distance.COSINE),
)

True

In [328]:
from qdrant_client.http import models

In [344]:
def minibatch(*tensors, **kwargs):

    batch_size = kwargs['batch_size']

    if len(tensors) == 1:
        tensor = tensors[0]
        for i in range(0, len(tensor), batch_size):
            yield tensor[i:i + batch_size]
    else:
        for i in range(0, len(tensors[0]), batch_size):
            yield tuple(x[i:i + batch_size] for x in tensors)

Delete collection

In [337]:
# client.delete_collection(collection_name="test_0")

False

In [345]:
for (minibatch_num,(idb,outputb)) in enumerate(minibatch(id,outputs,batch_size=32)):
    client.upsert(
    collection_name="test_0",
    points = models.Batch(
        ids = idb,
        vectors = outputb.tolist()
    )
)

In [346]:
client.get_collection(collection_name="test_0")

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=0, points_count=782, segments_count=8, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=3556, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema=

In [164]:
vector_2_query[4]

4055

In [169]:
from scipy.spatial.distance import cosine

similarity_score = 1 - cosine(outputs[0], outputs[3]) 
similarity_score

0.3244211971759796

In [353]:
hits = client.search(
    collection_name="test_0",
    query_vector=outputs[6].tolist(),
    limit=100,
)
for hit in hits:
    print(f"Hit {hit.id} Hit score: {hit.score}")

Hit 6 Hit score: 1.0
Hit 612 Hit score: 0.8765817
Hit 589 Hit score: 0.8756827
Hit 743 Hit score: 0.8665322
Hit 773 Hit score: 0.8609001
Hit 669 Hit score: 0.858551
Hit 501 Hit score: 0.84789
Hit 458 Hit score: 0.8390527
Hit 573 Hit score: 0.8356539
Hit 381 Hit score: 0.833912
Hit 779 Hit score: 0.8316698
Hit 33 Hit score: 0.82837373
Hit 493 Hit score: 0.8281207
Hit 604 Hit score: 0.82690436
Hit 161 Hit score: 0.82635224
Hit 129 Hit score: 0.82625586
Hit 409 Hit score: 0.8243472
Hit 364 Hit score: 0.82405794
Hit 264 Hit score: 0.8234814
Hit 388 Hit score: 0.82323575
Hit 602 Hit score: 0.8227576
Hit 273 Hit score: 0.8213948
Hit 92 Hit score: 0.8192987
Hit 530 Hit score: 0.81818795
Hit 162 Hit score: 0.8176499
Hit 552 Hit score: 0.8172264
Hit 90 Hit score: 0.81667507
Hit 520 Hit score: 0.81536514
Hit 80 Hit score: 0.81509376
Hit 750 Hit score: 0.81458104
Hit 300 Hit score: 0.813901
Hit 272 Hit score: 0.8129398
Hit 500 Hit score: 0.81285715
Hit 196 Hit score: 0.81195533
Hit 41 Hit score: 

In [354]:
outputs[0].size()

torch.Size([3556])

In [355]:
outputs.size(0)

782

In [356]:
similarity_dict = {}

Search top 100 similar vector outputs

In [357]:
for i in range(outputs.size(0)):
    hits = client.search(
    collection_name="test_0",
    query_vector=outputs[i].tolist(),
    limit=100,
    )
    similarity_dict[i] = [hit.id for hit in hits]

In [359]:
len(similarity_dict)

782

In [360]:
outputs.size()

torch.Size([782, 3556])

In [361]:
logits.size()

torch.Size([782, 2])

In [362]:
pred_label = torch.argmax(logits,dim = -1)

In [363]:
pred_label = pred_label.tolist()

In [365]:
for i in range(len(similarity_dict)):
    predict = ""
    real = ""
    for j in range(1,10):
        predict+= str(pred_label[similarity_dict[i][j]])
        predict+= "  "
        real+= str(pair_labels[similarity_dict[i][j]])
        real+= "  "
    print(f"Real label:{pred_label[i]} Neighbor -- Predict Labels: {predict} -- Real Labels: {real}")


Real label:0 Neighbor -- Predict Labels: 0  0  0  0  0  0  0  0  0   -- Real Labels: 0  0  1  0  0  0  0  0  0  
Real label:0 Neighbor -- Predict Labels: 0  0  0  0  0  0  0  0  0   -- Real Labels: 0  0  0  0  0  0  0  0  0  
Real label:0 Neighbor -- Predict Labels: 0  0  0  0  0  0  0  1  0   -- Real Labels: 0  0  0  0  0  0  1  0  0  
Real label:1 Neighbor -- Predict Labels: 1  1  1  1  1  0  1  1  1   -- Real Labels: 1  1  1  1  0  0  1  0  1  
Real label:0 Neighbor -- Predict Labels: 0  0  0  0  0  0  0  0  0   -- Real Labels: 0  0  0  0  0  0  0  0  0  
Real label:0 Neighbor -- Predict Labels: 0  0  0  0  0  0  0  0  0   -- Real Labels: 0  0  0  0  0  0  0  0  0  
Real label:0 Neighbor -- Predict Labels: 0  0  0  0  0  0  0  0  0   -- Real Labels: 0  0  0  0  0  0  0  0  0  
Real label:1 Neighbor -- Predict Labels: 1  1  1  1  0  0  0  1  1   -- Real Labels: 1  1  1  0  1  1  0  0  1  
Real label:0 Neighbor -- Predict Labels: 0  0  0  0  0  0  0  0  0   -- Real Labels: 0  0  0  0 

In [378]:
len(similarity_dict)

782

In [152]:
test = torch.randn(10, 16)

In [153]:
test.size()

torch.Size([10, 16])

In [ ]:
outputs
logits
vector_2_query
query_2_vector
similarity_dict

In [379]:
def get_new_batch(batch_id_claims): # return edge index of graph
    new_batch =[]
    for i,id in enumerate(batch_id_claims): #batch_id.: id of vector
        new_batch.append(id)
        new_batch = new_batch+ similarity_dict[id][:10]
    return new_batch

    

In [380]:
new_batch =  get_new_batch([0,1,2,3,4,5,6,7,8,9])

In [381]:
len(new_batch)

110

In [382]:
output = outputs[new_batch]

In [383]:
adj_matrix = output@output.T

In [384]:
adj_matrix.size()

torch.Size([110, 110])

In [385]:
adj_matrix[1]

tensor([22.9251, 22.9251, 18.1522, 21.2530, 16.5419, 16.8227, 16.8418, 14.7267,
        16.7862, 14.7796, 16.6267, 10.8893, 10.8893, 14.0285, 18.3824, 13.8483,
        12.6004, 20.6512, 13.0050, 16.7670, 10.7770, 10.9426,  4.2096,  4.2096,
         9.0254,  8.6805,  7.5657,  9.5553,  8.6765,  8.3450,  3.4646,  4.2429,
        10.1731,  4.0958,  4.0958,  5.0876,  5.9044,  4.0253,  5.6676,  3.6569,
         5.1906,  4.9169,  4.6360,  4.2135, 13.0285, 13.0285, 12.9824, 16.7670,
        13.8483, 12.6004, 14.5899, 14.2785, 20.6512, 13.1940, 15.1759,  9.7558,
         9.7558, 16.7862,  9.4593,  8.4654, 16.8227, 13.5843, 14.6776, 17.1621,
        18.4812, 14.4729, 21.6979, 21.6979, 18.0151, 23.6391, 23.3777, 23.9574,
        21.5364, 20.4258, 18.8795, 20.4211, 21.9522,  3.8952,  3.8952,  3.1554,
         4.0964,  4.6544,  3.6569,  3.4427,  7.6690, 12.0725,  6.3388,  5.9044,
        12.9232, 12.9232, 21.6666, 16.7670, 15.3130, 19.1406, 17.4464, 15.2180,
        23.6717, 20.3492, 23.1696,  5.71

In [406]:
from scipy.spatial.distance import cosine

similarity_score = 1 - cosine(outputs[0], outputs[5]) 
similarity_score

0.4477483928203583

In [387]:
def get_graph(new_batch_id,threshold,outputs,logits):
    output = outputs[new_batch_id]
    logit = logits[new_batch_id]
    adjacency_matrix = np.zeros((len(new_batch_id), len(new_batch_id)))
    for i in range(len(new_batch_id)):
        for j in range(len(new_batch_id)):
           adjacency_matrix[i][j] =   1 - cosine(output[i], output[j]) 
    edge_index = []
    for i in range(len(new_batch_id)):
        for j in range(len(new_batch_id)):
            if adjacency_matrix[i,j] >threshold:
                edge_index.append([i,j])
    return logit, torch.tensor(np.array(edge_index).transpose()).to(torch.int)
    

In [388]:
x,edge_index = get_graph(new_batch,0.8)

In [389]:
edge_index.shape

torch.Size([2, 1324])

In [376]:
import torch_geometric as tg
from torch_geometric.nn.conv import GATConv, GATv2Conv
from torch_geometric.data import Data
import torch.nn.functional as F
import torch.nn as nn

In [393]:

class PropagationModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.gat_conv = GATv2Conv(2, 2, heads=3, edge_dim=1)
        self.linear = nn.Linear(6,2)
    def forward(self,x,edge_index):
        x = self.gat_conv(x,edge_index)
        x = self.linear(x)
        return x
    def predict(self,x,edge_index):
        self.train(False)
        x=  self(x,edge_index)
        probs = F.softmax(x,dim=-1)
        return probs


In [408]:
np.array(edge_index)

array([[  0,   0,   1, ..., 109, 109, 109],
       [  0,   1,   0, ..., 106, 108, 109]], dtype=int32)

In [41]:
testModel = PropagationModel()

NameError: name 'PropagationModel' is not defined

In [40]:
probs = testModel.predict(x,edge_index)

NameError: name 'testModel' is not defined

In [5]:
from qdrant_client.models import Distance, VectorParams
from qdrant_client.http import models
from qdrant_client import QdrantClient
import torch
from scipy.spatial.distance import cosine

In [8]:
client = QdrantClient(url="http://localhost:6333")

In [14]:
def minibatch(*tensors, **kwargs):

    batch_size = kwargs['batch_size']

    if len(tensors) == 1:
        tensor = tensors[0]
        for i in range(0, len(tensor), batch_size):
            yield tensor[i:i + batch_size]
    else:
        for i in range(0, len(tensors[0]), batch_size):
            yield tuple(x[i:i + batch_size] for x in tensors)

In [1]:
def similarity_dict(name_data):
    name_outputs = f'/Users/namle/Desktop/GET/embed_probagation/output_{name_data}.pt'
    outputs = torch.load(name_outputs)
    client = QdrantClient(url="http://localhost:6333")
    client.create_collection(
        collection_name=name_data,
        vectors_config=VectorParams(size=outputs.size(1), distance=Distance.COSINE),
    )
    id=list(range(outputs.size(0)))
    for (minibatch_num,(idb,outputb)) in enumerate(minibatch(id,outputs,batch_size=32)):
        client.upsert(
            collection_name=name_data,
            points = models.Batch(
                ids = idb,
                vectors = outputb.tolist()
            )
        )
    similarity_dict = {}
    for i in range(outputs.size(0)):
        hits = client.search(
        collection_name=name_data,
        query_vector=outputs[i].tolist(),
        limit=10,
        )
        similarity_dict[i] = [hit.id for hit in hits]
    dict_name = f'embed_probagation/similar_dict_{name_data}.pt'
    torch.save(similarity_dict,dict_name)

def similar_main_snope():
    for i in range(5):
        name = f'snope_train_{i}'
        similarity_dict(name)
        name = f'snope_test_{i}'
        similarity_dict(name)
        if i ==4:
            name = f'snope_valid'
            similarity_dict(name)

In [30]:
similar_main_snope()

In [29]:
client.delete_collection(collection_name="snope_train_0")

True

In [25]:
name_data = "snope_train_0"

In [26]:
name_outputs = f'/Users/namle/Desktop/GET/embed_probagation/output_{name_data}.pt'
outputs = torch.load(name_outputs)
client = QdrantClient(url="http://localhost:6333")
client.create_collection(
    collection_name=name_data,
    vectors_config=VectorParams(size=outputs.size(1), distance=Distance.COSINE),
)
id=list(range(outputs.size(0)))
for (minibatch_num,(idb,outputb)) in enumerate(minibatch(id,outputs,batch_size=32)):
    client.upsert(
        collection_name=name_data,
        points = models.Batch(
            ids = idb,
            vectors = outputb.tolist()
        )
    )

In [21]:
outputs.size()

torch.Size([3126, 3556])

In [28]:
client.get_collection(collection_name="snope_train_0")

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=0, points_count=3126, segments_count=8, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=3556, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema

In [31]:
similar_dict = torch.load("/Users/namle/Desktop/GET/embed_probagation/similar_dict_snope_test_2.pt")

In [32]:
similar_dict

{0: [0, 763, 717, 689, 231, 289, 127, 307, 590, 364],
 1: [1, 289, 322, 646, 9, 510, 231, 775, 349, 500],
 2: [2, 646, 0, 298, 717, 1, 307, 322, 642, 664],
 3: [3, 738, 54, 735, 107, 661, 99, 530, 693, 193],
 4: [4, 653, 28, 604, 576, 602, 668, 680, 579, 642],
 5: [5, 632, 604, 510, 322, 579, 739, 653, 314, 646],
 6: [6, 343, 98, 354, 561, 213, 340, 761, 344, 359],
 7: [7, 582, 592, 392, 396, 684, 322, 14, 730, 245],
 8: [8, 201, 312, 765, 127, 317, 750, 472, 781, 490],
 9: [9, 322, 646, 1, 510, 582, 289, 231, 315, 551],
 10: [10, 643, 461, 18, 586, 637, 636, 538, 599, 557],
 11: [11, 738, 735, 661, 530, 587, 197, 667, 366, 730],
 12: [12, 595, 591, 395, 551, 610, 678, 581, 196, 597],
 13: [13, 580, 586, 660, 653, 20, 258, 540, 184, 112],
 14: [14, 398, 201, 337, 582, 331, 216, 548, 588, 595],
 15: [15, 263, 383, 317, 39, 54, 196, 710, 146, 373],
 16: [16, 148, 603, 694, 605, 34, 207, 264, 216, 244],
 17: [17, 520, 232, 195, 532, 26, 437, 140, 10, 112],
 18: [18, 662, 10, 586, 511, 648

In [11]:
from new_model import PropagationModel,FittingModel
import torch_utils as my_utils

In [12]:
dataset="Snopes"
fixed_length_left=30
fixed_length_right=100
log="logs/testget"
loss_type="cross_entropy"
batch_size=32
num_folds=5
use_claim_source=0
use_article_source=1
path="formatted_data/declare/"
hidden_size=300
epochs=100
num_att_heads_for_words=5
num_att_heads_for_evds=2
gnn_window_size=3
lr=0.0001
gnn_dropout=0.2
seed=123456
gsl_rate=0.6
fixed_length_left_src_chars = 20
fixed_length_right_src_chars = 20
seed = 123456
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [13]:
dataset="Snopes"
fixed_length_left=30
fixed_length_right=100
log="logs/get"
batch_size=32
gnn_window_size=3
fold_idx=0
if not os.path.exists(log):
      os.mkdir(log)
secondary_log_folder = os.path.join(log, "log_results_%s" % (dataset))
if not os.path.exists(secondary_log_folder):
  os.mkdir(secondary_log_folder)
secondary_log_folder = secondary_log_folder
root = os.path.join(os.path.join(path,dataset), "mapped_data")
tx = time.time()
kfold_dev_results, kfold_test_results = [], []
list_metrics = KeyWordSettings.CLS_METRICS
outfolder_per_fold = os.path.join(secondary_log_folder, "Fold_%s" % fold_idx)
if not os.path.exists(outfolder_per_fold):
    os.mkdir(outfolder_per_fold)
logfolder_result_per_fold = os.path.join(outfolder_per_fold, "result_%s.txt" % int(seed))

In [14]:
preprocessor = mz.preprocessors.CharManPreprocessor(fixed_length_left = fixed_length_left,
                                                    fixed_length_right = fixed_length_right,
                                                    fixed_length_left_src = fixed_length_left_src_chars,
                                                    fixed_length_right_src = fixed_length_right_src_chars)

In [15]:
dataset="Snopes"
fixed_length_left=30
fixed_length_right=100
log="logs/get"
batch_size=32
gnn_window_size=3
fold_idx=0
if not os.path.exists(log):
      os.mkdir(log)
secondary_log_folder = os.path.join(log, "log_results_%s" % (dataset))
if not os.path.exists(secondary_log_folder):
  os.mkdir(secondary_log_folder)
secondary_log_folder = secondary_log_folder
root = os.path.join(os.path.join(path,dataset), "mapped_data")
tx = time.time()
kfold_dev_results, kfold_test_results = [], []
list_metrics = KeyWordSettings.CLS_METRICS
outfolder_per_fold = os.path.join(secondary_log_folder, "Fold_%s" % fold_idx)
if not os.path.exists(outfolder_per_fold):
    os.mkdir(outfolder_per_fold)
logfolder_result_per_fold = os.path.join(outfolder_per_fold, "result_%s.txt" % int(seed))
file_handler = FileHandlerFC()
file_handler.init_log_files(logfolder_result_per_fold)
# root ="/content/drive/MyDrive/GET/formatted_data/declare/Snopes/mapped_data"
# ------------------------------------------------------------------------------------
predict_pack = cls_load_data.load_data(root + "/%sfold" % num_folds,  'test_%s' % 0, kfolds = num_folds)
train_pack = cls_load_data.load_data(root + "/%sfold" % num_folds, 'train_%sres' % 0, kfolds = num_folds)
# predict_pack = cls_load_data.load_data(root + "/%sfold" % num_folds,  'testnew' , kfolds = num_folds)
# train_pack = cls_load_data.load_data(root + "/%sfold" % num_folds, 'trainnew' , kfolds = num_folds)
valid_pack = cls_load_data.load_data(root, 'dev', kfolds = num_folds)

a = train_pack.left["text_left"].str.lower().str.split().apply(len).max()
b = valid_pack.left["text_left"].str.lower().str.split().apply(len).max()
c = predict_pack.left["text_left"].str.lower().str.split().apply(len).max()
max_query_length = max([a, b, c])
min_query_length = min([a, b, c])

a = train_pack.right["text_right"].str.lower().str.split().apply(len).max()
b = valid_pack.right["text_right"].str.lower().str.split().apply(len).max()
c = predict_pack.right["text_right"].str.lower().str.split().apply(len).max()
max_doc_length = max([a, b, c])
min_doc_length = min([a, b, c])

file_handler.myprint("Min query length, " + str(min_query_length) + " Min doc length " + str(min_doc_length))
file_handler.myprint("Max query length, " + str(max_query_length) + " Max doc length " + str(max_doc_length))
global additional_data
additional_data = {KeyWordSettings.OutputHandlerFactChecking: file_handler,
                    KeyWordSettings.GNN_Window: gnn_window_size}

print('parsing data')

train_processed = preprocessor.fit_transform(train_pack)  # This is a DataPack
valid_processed = preprocessor.transform(valid_pack)
predict_processed = preprocessor.transform(predict_pack)


train_interactions = ClassificationInteractions(train_processed, **additional_data)
valid_interactions = ClassificationInteractions(valid_processed, **additional_data)
predict_interactions = ClassificationInteractions(predict_processed, **additional_data)
file_handler.myprint('done extracting')
  

match_model = PropagationModel()

file_handler.myprint("Fitting Model")

data = "snope" if dataset== "Snopes" else "poli"
match_model = PropagationModel()
fit_model = FittingModel(net = match_model,batch_size=batch_size, n_iter = epochs,idx_fold=0,
                            threshold=0.8,data=data,output_handler_fact_checking = file_handler, 
                            use_cuda = 0,logfolder=outfolder_per_fold,seed=seed,early_stopping= 11)

Processing text_right with transform: 100%|██████████| 2756/2756 [00:00<00:00, 647700.00it/s]


Min query length, 22 Min doc length 100
Max query length, 28 Max doc length 100
parsing data


Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 3126/3126 [00:00<00:00, 11171.95it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 21074/21074 [00:07<00:00, 2971.42it/s]
Processing text_right with extend: 100%|██████████| 21074/21074 [00:00<00:00, 176654.69it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 3126/3126 [00:00<00:00, 11097.00it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 21074/21074 [00:07<00:00, 2832.39it/s]
Processing text_right with transform: 100%|██████████| 2756/2756 [00:00<00:00, 170524.31it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 433/433 [00:00<00:00, 11742.67it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2756/2756 [00:00<00:

Converting DataFrame to Normal Dictionary of Data
[NOTICE] MatchZoo use queryID and docID as index in dataframe left and right, therefore, iterrows will return index which is left_id or right_id


/Users/namle/Library/CloudStorage/GoogleDrive-hcmutepfl@gmail.com/My Drive/GET/interactions.py:15: RuntimeWarning: divide by zero encountered in power
  d_inv_sqrt = np.power(rowsum, -0.5).flatten()


[NOTICE] MatchZoo use queryID and docID as index in dataframe left and right, therefore, iterrows will return index which is left_id or right_id
Converting DataFrame to Normal Dictionary of Data
[NOTICE] MatchZoo use queryID and docID as index in dataframe left and right, therefore, iterrows will return index which is left_id or right_id
[NOTICE] MatchZoo use queryID and docID as index in dataframe left and right, therefore, iterrows will return index which is left_id or right_id
Converting DataFrame to Normal Dictionary of Data
[NOTICE] MatchZoo use queryID and docID as index in dataframe left and right, therefore, iterrows will return index which is left_id or right_id
[NOTICE] MatchZoo use queryID and docID as index in dataframe left and right, therefore, iterrows will return index which is left_id or right_id
done extracting
Fitting Model
Using: <new_model.Loss_func object at 0x7fc191bca370>


In [12]:
fit_model.train_outputs.size()

torch.Size([3126, 3556])

In [13]:
query_ids,pair_labels = fit_model._sampler.get_instance_new(train_interactions,100)

In [15]:
query_ids.shape

(3126,)

In [71]:
query_2_vector = {} 
for i,id in enumerate(query_ids):
    query_2_vector[id] = i
id_b = np.array([query_2_vector[i] for i in query_ids ])
# id_b,query_ids, pair_labels = my_utils.shuffle(id_b,query_ids, pair_labels)
# all_labels = pair_labels.flatten().astype(int).tolist()

# pair_labels = pair_labels.tolist()
epoch_loss, total_pairs = 0.0, 0
t1 = time.time()
for (minibatch_num,
        (batch_id,batch_label)) \
        in enumerate(my_utils.minibatch(id_b,pair_labels, batch_size=29)):
    batch_label = my_utils.gpu(torch.from_numpy(batch_label), fit_model._use_cuda)
    batch_id = my_utils.gpu(torch.from_numpy(batch_id), fit_model._use_cuda)
    break

In [72]:
old_id,x,edge_index = fit_model.get_graph("train",batch_id,fit_model._threshold)

In [4]:
from torch_geometric.nn.conv import GATConv, GATv2Conv

In [5]:
class newModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.gat_conv = GATv2Conv(2, 2, heads=3, edge_dim=1)
        self.dropout = nn.Dropout(p=0.4) 
        self.linear = nn.Linear(6,2)

        self.linear.apply(my_utils.init_weights)
    def forward(self,x,edge_index,old_id):
        num_node = x.size(0)//11
        x = self.gat_conv(x,edge_index)
        x = self.dropout(x)
        x = F.relu(x)
        x = self.linear(x)
        x = x.view(num_node,11,2)
        x = torch.mean(x,dim=1)
        return x
    def predict(self,x,edge_index,old_id):
        self.train(False)
        x=  self(x,edge_index,old_id)
        probs = F.softmax(x,dim=-1)
        return probs

In [6]:
model = newModel()

NameError: name 'my_utils' is not defined

In [76]:
fit_model._net = model

In [77]:
logits = fit_model._net(x,edge_index,old_id)

In [78]:
logits.size()

torch.Size([29, 2])

In [36]:
x = torch.arange(352)

In [38]:
x = x.view(32,11)

In [41]:
x[1]


tensor([11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21])

In [47]:
import torch

# Original tensor (assuming batch size 32, 1 node with 10 neighbors, feature dim 2)
x = torch.randn(352, 2)

# Number of nodes per graph (excluding the central node)
num_neighbors = 10

# Calculate the number of graphs in the batch
num_graphs = x.size(0) // (1 + num_neighbors)  # Integer division for batch size

# Reshape to separate features for each node within a graph
x = x.view(num_graphs, 1 + num_neighbors, 2)

In [50]:

x_old = x
x_old.size()

torch.Size([32, 11, 2])

In [58]:
t = torch.mean(x_old[0],dim=0)

In [59]:
t

tensor([-0.1113,  0.3010])

In [51]:
x = torch.mean(x,dim=1)

In [52]:
x.size()

torch.Size([32, 2])

In [54]:
x[0]

tensor([-0.1113,  0.3010])

In [34]:
# Calculate average pool across nodes (excluding the central node)
pooled_x = torch.mean(x[:, 1:], dim=1)  # Skip the first node (central node)


In [35]:
pooled_x.size()

torch.Size([32, 2])

In [33]:
x.size()

torch.Size([32, 11, 2])